In [1]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import Perceptron

In [2]:
import pandas as pd

def readData():
    dftrain = pd.read_excel('data86/dataTrain.xlsx', sheet_name='dataTrain')
    dftest = pd.read_excel('data86/dataTest.xlsx', sheet_name='dataTest')
    
    comTrain = dftrain['comment'].values.tolist()
    y_train= dftrain['label'].values.tolist()
    comTest = dftest['comment'].values.tolist()
    y_test = dftest['label'].values.tolist()
    return [comTrain, y_train, comTest, y_test]

In [3]:
# làm sạch dữ liệu
import re
import pyvi
from pyvi import ViTokenizer 

def preprocess_reviews(reviews):
    REPLACE_NO_SPACE = re.compile("[.;:\',\"()\[\]]")#("[.;:!\'?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    reviews = [REPLACE_NO_SPACE.sub("", str(line).lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    reviews = [line.lstrip() for line in reviews]
    reviews = [ViTokenizer.tokenize(rev) for rev in reviews]
    return reviews


In [4]:
# vector hóa data
def vectorizer(com):
    cv = CountVectorizer(binary=True)
    cv.fit(comTrain)
    X_train = cv.transform(comTrain)
    X_test = cv.transform(com)
    return [X_train,X_test]

In [5]:
def SVM():
    clf = SVC(kernel = 'linear', C = 1e5) # just a big number 
    clf.fit(X_train.toarray(), y_train) 
    w = clf.coef_
    b = clf.intercept_
    #print('w = ', w)
    #print('b = ', b)
    print ("Final Accuracy SVM: %s"% accuracy_score(y_test, clf.predict(X_test.toarray())))
    #return clf

In [6]:
def multiNB():
    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)

    acc_test = accuracy_score(y_pred_test,y_test)
    acc_train = accuracy_score(y_pred_train,y_train)
    #print('accuracy_train: ', acc_train)
    print('Final Accuracy multi Naive Bayes: ', acc_test)

In [7]:
def perceptron():
    clf = Perceptron(tol=1e-3, random_state=0)
    clf.fit(X_train, y_train)
    acc_train=clf.score(X_train, y_train)
    acc_test =clf.score(X_test, y_test)
    #print('accurracy_train of folder {}: ', acc_train)
    print('Final Accuracy perceptron: ', acc_test)

In [8]:
# X_trainlog, X_testlog, y_trainlog, y_testlog = train_test_split(
#     X_train, y_train, train_size = 0.75
# )
def logistic():
#     for c in [0.01, 0.05, 0.25, 0.5, 1]:
#         lr = LogisticRegression(C=c)
#         lr.fit(X_trainlog, y_trainlog)
#         print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_testlog, lr.predict(X_testlog))))

    final_model = LogisticRegression(C=0.5)
    final_model.fit(X_train, y_train)
    print ("Final Accuracy logistic: %s"% accuracy_score(y_test, final_model.predict(X_test)))

In [9]:
[comTrain, y_train, comTest, y_test] = readData()
comTrain = preprocess_reviews(comTrain)
comTest = preprocess_reviews(comTest)
[X_train,X_test] = vectorizer(comTest)



In [12]:
multiNB()
logistic()
perceptron()
SVM()

Final Accuracy multi Naive Bayes:  0.8675324675324675


C:\Users\Thuy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Final Accuracy logistic: 0.8519480519480519
Final Accuracy perceptron:  0.8051948051948052
Final Accuracy SVM: 0.8025974025974026


In [13]:
def multiNBPre(comment):
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    y_pred_test = clf.predict(comment)
    return y_pred_test[0]
def predict(comment):
    [a,check] = vectorizer([comment])
    return multiNBPre(check)
predict('may nay khong tot')

1

In [14]:
from flask import Flask,url_for,request,redirect,render_template
app = Flask(__name__)

@app.route('/')
def welcome():
    return redirect('/home')

@app.route('/home',methods=['GET','POST'])
def home():
    res =""
    err = ""
    if request.method == 'POST':
        label = predict(request.form['comment'])
        if label == 1:
            res = 'Tích cực' # 'positive'
        else:
            res =   'Tiêu cực' #'negative'
    return render_template('home.html',result = res, error = err)

@app.route('/login',methods =['GET','POST'])
def login():
    print(request.method)
    error = ''
    if request.method =='POST':
        if request.form['username']!='admin':
            error = 'ERROR'
        else:
            return redirect(url_for('home'))
    return render_template('login.html',error=error)

if __name__=='__main__':
    app.run(host='localhost', port=7000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:7000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2020 08:39:45] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [03/Apr/2020 08:39:45] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 08:39:45] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Apr/2020 08:48:21] "POST /home HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 08:48:42] "POST /home HTTP/1.1" 200 -
